# Lead Scoring Classification

In [1]:
# importing libraries
import pandas as pd
import numpy as np

In [2]:
#read csv
df = pd.read_csv("./course_lead_scoring.csv")

In [3]:
# most frequent industry
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
education        187
healthcare       187
technology       179
manufacturing    174
Name: count, dtype: int64

In [5]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [6]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [9]:
columns = list(df.columns)
columns

['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score',
 'converted']

In [16]:
print(df['lead_score'].dtypes)

float64


In [20]:
for col in columns:
    if df[col].dtype=='float64' or df[col].dtype=='int64':
        df[col]=df[col].fillna(0.0)
    else:
        df[col]=df[col].fillna('NA')

In [21]:
df['']

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Correlation

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [25]:
df[['interaction_count','number_of_courses_viewed']].corrwith(df['lead_score'])

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [26]:
df[['number_of_courses_viewed','annual_income']].corrwith(df['interaction_count'])

number_of_courses_viewed   -0.023565
annual_income               0.048618
dtype: float64

## spliting of dataset

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
df_full_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train,df_val = train_test_split(df_full_train,test_size=0.25,random_state=42)

In [34]:
df_full_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)
df_train.reset_index(drop=True,inplace=True)
df_val.reset_index(drop=True,inplace=True)

In [36]:
y_full_train = df_full_train['converted']
y_test = df_test['converted']
y_train = df_train['converted']
y_val = df_train['converted']

del df_full_train['converted']
del df_test['converted']
del df_train['converted']
del df_val['converted']

## mutual score

In [38]:
from sklearn.metrics import mutual_info_score

In [39]:
def mutual_score(series):
    return mutual_info_score(series,y_train)

In [45]:
categorical = list(df_train.dtypes[df_train.dtypes=='object'].index)

In [50]:
df_train.isnull().sum()

lead_source                  89
industry                     78
number_of_courses_viewed      0
annual_income               110
employment_status            66
location                     38
interaction_count             0
lead_score                    0
dtype: int64

In [48]:
mi = df_train[categorical].apply(mutual_score)

ValueError: Input contains NaN